In [1]:
from math import log
from rocketcea.cea_obj_w_units import CEA_Obj

In [2]:
C = CEA_Obj(oxName="LOX", fuelName="RP-1")

Wpayload = 8500
Wstg = 5106
Wpropellant = 45920

Winit = Wstg + Wpropellant + Wpayload
Wfinal = Winit - Wpropellant

def show_deltaV( Pc=475.0, eps=84.0, MR=5.88 ):
    IspVac = C.get_Isp(Pc=Pc, MR=MR, eps=eps)
    IspDel = 0.969 * IspVac
    deltaV = 32.174 * IspDel * log( Winit / Wfinal ) # ft/sec
    print( '%8.1f %8.1f    %8.2f    %8.1f     %8.1f        %8.1f     '%(Pc, eps, MR, IspVac, IspDel, deltaV))

print(' Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)')
show_deltaV(Pc=475, eps=120)


 Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)
   475.0    120.0        5.88       322.1        312.2         14822.7     


# Missão

In [3]:
deltav_mission = 12000
payload_weight = 5000
payload_height = 6.7
payload_diameter = 4.6

# Parâmetros Fixos

In [4]:
n_stages = 2
deltav_stages = {1: 3500, 2: 8500}
stage_radii = {1: 1.8, 2: 1.8}
oxidizer = "LOX"
fuel = "RP-1"
engine_cycle = "Gas Generator"
chamber_pressure = {1: 97, 2: 97}
coef_estrutual = {1: 0.06, 2: 0.05}
max_n_engines = {1: 15, 2: 1}


# Fronteira dos parâmetros

In [5]:
mixture_ratior_border = {"begin": 1.5, "end": 3.5}
throat_diameter_border = {"begin": 0.2, "end": 0.3}

# Restrições

In [6]:
min_acceleration = {1: 1.3, 2: 0.8}
max_ld_ration = {1: 25, 2: 25}
molar_masses = {"LOX":32, "RP-1":175}

# -------

In [51]:
import math

class Engine:
    def __init__(self,
                 fuel_name:str,
                 ox_name:str,
                 mr:float,
                 pc:float,
                 nozzle_diam:float,
                 eps:float):
        """
        fuel_name: 
        ox_name (oxidizer name):
        mr (mixture ratio):
        nozzle_diam (nozzle throat diameter):
        esp (nozzle expansion ratio)
        """
        
        self.fuel_name = fuel_name
        self.ox_name = ox_name
        self.mr = mr
        self.pc = pc
        self.nozzle_diam = nozzle_diam
        self.eps = eps

        self.R_universal = 8.314462 # Constante universal dos gases

        self.C = CEA_Obj(oxName=self.ox_name, 
                         fuelName=self.fuel_name,
                         isp_units='sec',
                         cstar_units='m/s', 
                         pressure_units='bar', 
                         temperature_units='K', 
                         sonic_velocity_units='m/s', 
                         enthalpy_units='J/g', 
                         density_units='kg/m^3', 
                         specific_heat_units='J/kg-K', 
                         viscosity_units='millipoise', 
                         thermal_cond_units='mcal/cm-K-s')


    def get_F_vac(self):
        IspVac = self.C.get_Isp(Pc=self.pc, MR=self.mr, eps=self.eps)
        ChamberDensity = self.C.get_Chamber_Density(Pc=self.pc, MR=self.mr, eps=self.eps)
        SonicVel = self.C.get_Chamber_SonicVel(Pc=self.pc, MR=self.mr, eps=self.eps)
        SonicVels = self.C.get_SonicVelocities(Pc=self.pc, MR=self.mr, eps=self.eps)
        mach = self.C.get_MachNumber(Pc=self.pc, MR=self.mr, eps=self.eps)
        gamma = self.C.get_exit_MolWt_gamma(Pc=self.pc, MR=self.mr, eps=self.eps)[1]
        temperature_chamber, temperature_throat, temperature_exit = self.C.get_Temperatures(Pc=self.pc, MR=self.mr, eps=self.eps)
        moles = self.C.get_SpeciesMoleFractions(Pc=self.pc, MR=self.mr, eps=self.eps)
        avg_mol = 1/self.mr * molar_masses[self.fuel_name] + (1 - 1/self.mr) * molar_masses[self.ox_name]
        pc_pe = self.C.get_PcOvPe(Pc=self.pc, MR=self.mr, eps=self.eps)

        mass_flow = math.pi * self.eps * (self.nozzle_diam**2) * SonicVel * ChamberDensity
        F_vac = IspVac * mass_flow

        nozzle_throat_area = math.pi*(self.nozzle_diam/2 **2)
        nozzle_exit_area = nozzle_throat_area * self.eps
        pe = 1/pc_pe * self.pc

        R_fuel = self.R_universal / avg_mol * 1000  # Dividindo por 1000 para converter g em kg


        # Mach = 1 (choked)
        # 10e5 para converter 'bar' em 'Pa'
        mass_flow = (nozzle_throat_area * self.pc * (10e5) / (math.sqrt(temperature_chamber))) * \
                    math.sqrt(gamma/R_fuel) * \
                    ((gamma + 1)/2) ** \
                    (- (gamma + 1)/(2*(gamma - 1)))

        # Mach =!= 1
        # Pesquisar sobre choked 
        #mass_flow = (nozzle_throat_area * self.pc (10e5) / (math.sqrt(temperature_chamber))) * \
        #            math.sqrt(gamma/R_fuel) * mach * \
        #            (1 + ((gamma + 1)/2) * mach**2) ** \
        #            (- (gamma + 1)/(2*(gamma - 1)))


        

        print(mass_flow)
        print(self.pc, pe)
        print(temperature_chamber)

        return F_vac



engine = Engine(fuel_name="RP-1", 
                ox_name="LOX",
                mr=2.9,
                pc=97,
                nozzle_diam=0.25,
                eps=165)

engine.get_F_vac()



19900.073124247043
97 0.04554990123384743
3756.9659075129844


113448876.85503739

# Referências bibliográficas

- https://cearun.grc.nasa.gov/intro.html
- https://www.grc.nasa.gov/www/k-12/airplane/mflchk.html
- https://rocketcea.readthedocs.io/en/latest/functions.html
- http://www.braeunig.us/space/index.htm
- https://www.grc.nasa.gov/www/k-12/airplane/rktthsum.html


